# Preprocessing Encoder, Pipeline, SequentialEncoder and FeatureUnion example

The easyflow.preprocessing module contains functionality similar to what sklearn does with its Pipeline, FeatureUnion and ColumnTransformer does. 

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Normalization, CategoryEncoding, StringLookup

In [2]:
# local imports
from easyflow.data.mapper import TensorflowDataMapper
from easyflow.preprocessing.preprocessor import Encoder, Pipeline, SequentialEncoder, FeatureUnion
from easyflow.preprocessing.custom import IdentityPreprocessingLayer

## Read in data and map as tf.data.Dataset

Use the TensorflowDataMapper class to map pandas data frame to a tf.data.Dataset type

In [3]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
dataframe = pd.read_csv(file_url)
dataframe = dataframe.copy()
labels = dataframe.pop("target")

batch_size = 32
dataset_mapper = TensorflowDataMapper() 
dataset = dataset_mapper.map(dataframe, labels)
train_data_set, val_data_set = dataset_mapper.split_data_set(dataset)
train_data_set = train_data_set.batch(batch_size)
val_data_set = val_data_set.batch(batch_size)

In [4]:
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal


## Set constants

In [5]:
NUMERICAL_FEATURES = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope']
CATEGORICAL_FEATURES = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'ca']
# thal is represented as a string
STRING_CATEGORICAL_FEATURES = ['thal']

## Setup Preprocessing layer using FeatureUnion

Use Encoder and SequentialEncoder to preprocess features by putting everything in a FeatureUnion object

In [6]:
feature_encoder_list = [
                        Encoder([('numeric_encoder', Normalization, NUMERICAL_FEATURES)]),
                        Encoder([('categorical_encoder', CategoryEncoding, CATEGORICAL_FEATURES)]),
                        # For feature thal we first need to run StringLookup followed by a CategoryEncoding layer
                        SequentialEncoder([('string_encoder', StringLookup, STRING_CATEGORICAL_FEATURES),
                                           ('categorical_encoder', CategoryEncoding, STRING_CATEGORICAL_FEATURES)])
                        ]


In [7]:
encoder = FeatureUnion(feature_encoder_list)
all_feature_inputs, preprocessing_layer = encoder.encode(dataset)

In [8]:
print(preprocessing_layer)

Tensor("concatenate/concat:0", shape=(None, 31), dtype=float32)


In [9]:
# setup simple network
x = tf.keras.layers.Dense(128, activation="relu")(preprocessing_layer)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=all_feature_inputs, outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'), tf.keras.metrics.AUC(name='auc')])

tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")


('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [10]:
history=model.fit(train_data_set, validation_data=val_data_set, epochs=10)

Epoch 1/10
8/8 [==============================] - 0s 54ms/step - loss: 0.6361 - accuracy: 0.6123 - auc: 0.6322 - val_loss: 0.5864 - val_accuracy: 0.7368 - val_auc: 0.7014
Epoch 2/10
8/8 [==============================] - 0s 5ms/step - loss: 0.5796 - accuracy: 0.7048 - auc: 0.6562 - val_loss: 0.5326 - val_accuracy: 0.7237 - val_auc: 0.8596
Epoch 3/10
8/8 [==============================] - 0s 5ms/step - loss: 0.5599 - accuracy: 0.7181 - auc: 0.7032 - val_loss: 0.4977 - val_accuracy: 0.7763 - val_auc: 0.8385
Epoch 4/10
8/8 [==============================] - 0s 5ms/step - loss: 0.4835 - accuracy: 0.7974 - auc: 0.7897 - val_loss: 0.4257 - val_accuracy: 0.8026 - val_auc: 0.9130
Epoch 5/10
8/8 [==============================] - 0s 5ms/step - loss: 0.4673 - accuracy: 0.8018 - auc: 0.8220 - val_loss: 0.4792 - val_accuracy: 0.7763 - val_auc: 0.8417
Epoch 6/10
8/8 [==============================] - 0s 5ms/step - loss: 0.4136 - accuracy: 0.8150 - auc: 0.8655 - val_loss: 0.3951 - val_accuracy: 0.81

## More flexibility with Pipeline

FeatureUnion subclasses Pipeline and concatenates(i.e union) the layers. For more flexibility like a wide and deep neural network, Pipeline class will give you more flexibility in that sense

In [11]:
feature_encoder_list = [
                        Encoder([('numeric_encoder', Normalization, NUMERICAL_FEATURES)]),
                        Encoder([('categorical_encoder', CategoryEncoding, CATEGORICAL_FEATURES)]),
                        # For feature thal we first need to run StringLookup followed by a CategoryEncoding layer
                        SequentialEncoder([('string_encoder', StringLookup, STRING_CATEGORICAL_FEATURES),
                                           ('categorical_encoder', CategoryEncoding, STRING_CATEGORICAL_FEATURES)])
                        ]

encoder = Pipeline(feature_encoder_list)
all_feature_inputs1, preprocessing_layer1 = encoder.encode(dataset)

In [12]:
print(preprocessing_layer)

Tensor("concatenate/concat:0", shape=(None, 31), dtype=float32)
